<img style="float: left;;" src='Figures/alinco.png' /></a>

# Modulo II: Traducción automática y Local Sensitive Hashing (LSH)


Ahora implementaremos un sistema de traducción automática y luego
veremos cómo funcionan las funciones hash. Empecemos importando
las funciones requeridas!


```
nltk.download('stopwords')
nltk.download('twitter_samples')
```

In [1]:
# importar las librerías


# Embeddings para palabras en Inglés y Frances

Escribe un programa que traduzca del inglés al francés.

## Los datos

El conjunto de datos completo para las wordembeddings en inglés es de aproximadamente 3,64 gigabytes, y el francés
son de aproximadamente 629 megabytes. Trabajaremos con un conjunto más pequeño de datos, una muestra significativa.


#### Subconjunto de los datos




#### Cargue dos diccionarios que mapean las palabras del inglés al francés
* Un diccionario de entrenamiento
* y un diccionario de prueba.

#### Viendo los diccionarios de Inglés y Francés

* `en_fr_train` es un diccionario donde el key es la palabra en inglés y el valor es la palabra en francés.
```
{'the': 'la',
 'and': 'et',
 'was': 'était',
 'for': 'pour',
```

* `en_fr_test` es similar que `en_fr_train`, pero este es un conjunto de prueba. 

## Generando matrices de embeddings y transformación

####  Traducción de diccionario de inglés a francés mediante embeddings

Implementaremos la función `get_matrices`, que toma los datos cargados y retorna las matrices `X` y `Y`.

Entrada:
- `en_fr` : diccionario del Inglés a Francés
- `en_embeddings` : embeddings de palabras en inglés
- `fr_embeddings` : embeddings de palabras en Francés

Retorna:
- matrices `X` y `Y`, donde cada renglón en X es la palabra embebida para una plabra en inglés, y lo mismo con Y es la palabra embebida en francés.

<div style="width:image width px; font-size:100%; text-align:center;">
<img src='Figures/X_to_Y.jpg' alt="alternate text" width="width" height="height" style="width:800px;height:200px;" /> Figure 2 </div>

Utilice el diccionario `en_fr` para asegurarse de que la i-ésima fila de la matriz` X`
corresponde a la i-ésima fila de la matriz "Y".


Ahora usaremos la función `get_matrices ()` para obtener los conjuntos `X_train` e` Y_train`
de los embeddings de palabras en inglés y francés.



# Traductores

<div style="width:image width px; font-size:100%; text-align:center;"><img src='Figures/e_to_f.jpg' alt="alternate text" width="width" height="height" style="width:700px;height:200px;" />  </div>

Escriba un programa que traduzca palabras del inglés al francés utilizando word embeddings y modelos de espacio vectorial.


##  Traducción como transformación lineal de embeddings
Dados los diccionarios de embeddings de palabras en inglés y francés, crearemos una matriz de transformación `R`
* Dada una palabra incrustada en inglés, $ \mathbf {e} $, puede multiplicar $ \mathbf {eR} $ para obtener una nueva palabra incrustada $ \mathbf {f} $.

    
* A continuación, puede calcular los vecinos más cercanos a `f` en los embeddings francesas y recomendar la palabra que es más similar a los embeddings de palabras transformadas.

### Describir la traducción como el problema de minimización

Encuentre una matriz `R` que minimice la siguiente ecuación.

$$\arg \min _{\mathbf{R}}\| \mathbf{X R} - \mathbf{Y}\|_{F}\tag{1} $$

### Norma de Frobenius

La norma de Frobenius de una matriz $ A $ (asumiendo que es de dimensión $ m, n $) se define como la raíz cuadrada de la suma de los cuadrados absolutos de sus elementos:

$$\|\mathbf{A}\|_{F} \equiv \sqrt{\sum_{i=1}^{m} \sum_{j=1}^{n}\left|a_{i j}\right|^{2}}\tag{2}$$

### Función de perdida

En las aplicaciones del mundo real, la pérdida de la norma Frobenius:

$$\| \mathbf{XR} - \mathbf{Y}\|_{F}$$

a menudo se reemplaza por su valor al cuadrado dividido por $ m $:

$$ \frac{1}{m} \|  \mathbf{X R} - \mathbf{Y} \|_{F}^{2}$$

donde $ m $ es el número de ejemplos (filas en $ \ mathbf {X} $).

* Se encuentra la misma R cuando se usa esta función de pérdida en comparación con la norma de Frobenius original.
* La razón para tomar el cuadrado es que es más fácil calcular el gradiente del Frobenius al cuadrado.
* La razón para dividir entre $ m $ es que estamos más interesados en la pérdida promedio por inserción que en la pérdida de todo el conjunto de entrenamiento.
     * La pérdida de todo el conjunto de entrenamiento aumenta con más palabras (ejemplos de entrenamiento),
     por lo que tomar el promedio nos ayuda a rastrear la pérdida promedio independientemente del tamaño del conjunto de entrenamiento.



### Implementación del mecanismo de traducción

#### Calculando el loss
* La función de pérdida será la norma de Frobenoius al cuadrado de la diferencia entre
matriz y su aproximación, dividida por el número de ejemplos de entrenamiento $ m $.
* Su fórmula es:
$$ L(X, Y, R)=\frac{1}{m}\sum_{i=1}^{m} \sum_{j=1}^{n}\left( a_{i j} \right)^{2}$$

donde $a_{i j}$ es el valo de la $i$-ésimo renglón y $j$-ésima columna de la matriz $\mathbf{XR}-\mathbf{Y}$.

* Calcular la aproximación de `Y` mediante la matriz multiplicando` X` y `R`
* Calcular la diferencia `XR - Y`
* Calcular la norma de Frobenius al cuadrado de la diferencia y divídala por $ m $.


### Calculando el gradiente de la función loss con respecto a la matriz de transforación R

* Calcular el gradiente de la pérdida con respecto a la matriz de transformación "R".
* El gradiente nos da la dirección en la que debemos disminuir `R`
para minimizar la pérdida.
* $ m $ es el número de ejemplos de entrenamiento (número de filas en $ X $).
* La fórmula para el gradiente de la función de pérdida $ 𝐿 (𝑋, 𝑌, 𝑅) $ es:

$$\frac{d}{dR}𝐿(𝑋,𝑌,𝑅)=\frac{d}{dR}\Big(\frac{1}{m}\| X R -Y\|_{F}^{2}\Big) = \frac{2}{m}X^{T} (X R - Y)$$



### Encontrar la R óptima con el algoritmo de descenso de gradiente

#### Gradient descent


Pseudocódigo:
1. Calcular el gradiente $g$ del loss con respecto a la matriz $R$.
2. Update $R$ con la formula:
$$R_{\text{new}}= R_{\text{old}}-\alpha g$$

Donde $\alpha$ es el learning rate, que es un escalar.

#### Learning rate

* La tasa de aprendizaje o "tamaño de paso" $ \ alpha $ es un coeficiente que decide cuánto queremos cambiar $ R $ en cada paso.
* Si cambiamos $ R $ demasiado, podríamos saltarnos el óptimo dando un paso demasiado grande.
* Si solo hacemos pequeños cambios en $ R $, necesitaremos muchos pasos para alcanzar el óptimo.
* La tasa de aprendizaje $ \ alpha $ se usa para controlar esos cambios.
* Los valores de $ \ alpha $ se eligen dependiendo del problema, y usaremos `learning_rate` $ = 0.0003 $ como valor predeterminado para nuestro algoritmo.

## Clacular la matriz de transformación R



## Probando el traductor

### Algoritmo k-Nearest neighbors 

[k-Nearest neighbors algorithm](https://en.wikipedia.org/wiki/K-nearest_neighbors_algorithm) 
* k-NN es un método que toma un vector como entrada y encuentra los otros vectores en el conjunto de datos que están más cerca de él.
* La 'k' es el número de "vecinos más cercanos" a encontrar (por ejemplo, k = 2 encuentra los dos vecinos más cercanos).

### Buscando el embedding de la traducción 
Dado que estamos aproximando la función de traducción de los embeddings de inglés a francés mediante una matriz de transformación lineal $ \mathbf {R} $, la mayoría de las veces no obtendremos la incrustación exacta de una palabra francesa cuando transformamos los embeddings $ \mathbf { e} $ de alguna palabra en inglés en particular en el espacio de embeddings francés.
* ¡Aquí es donde $ k $ -NN se vuelve realmente útil! Al usar $ 1 $ -NN con $ \mathbf {eR} $ como entrada, podemos buscar un embedding $ \mathbf {f} $ (como una fila) en la matriz $ \mathbf {Y} $ que es la más cercana a el vector transformado $ \mathbf {eR} $

### Similaridad por  Coseno
Similitud de coseno entre los vectores $ u $ y $ v $ calculada como el coseno del ángulo entre ellos.
La formula es
$$\cos(u,v)=\frac{u\cdot v}{\left\|u\right\|\left\|v\right\|}$$

* $\cos(u,v)$ = $1$ cuando $u$ y $v$ se encuentran en la misma línea y tienen la misma dirección.
* $\cos(u,v)$ es $-1$ Cuando ellas tienen direcciones exactamente opuestas.
* $\cos(u,v)$ es $0$ cuando los vectores son ortogonales (perpendiculares) entre sí.

#### Nota: La distancia y la similitud son cosas bastante opuestas.
* Podemos obtener la métrica de la distancia a partir de la similitud del coseno, pero la similitud del coseno no se puede usar directamente como la métrica de la distancia.
* Cuando la similitud del coseno aumenta (hacia $ 1 $), la "distancia" entre los dos vectores disminuye (hacia $ 0 $).
* Podemos definir la distancia del coseno entre $ u $ y $ v $ como
$$ d_{\text {cos}} (u, v) = 1- \cos(u, v) $$

Completando la función `nearest_neighbor()`:



### Test your translation and compute its accuracy

* Calculando el accuracy como $$\text{accuracy}=\frac{\#(\text{correct predictions})}{\#(\text{total predictions})}$$


# LSH y búsqueda de documentos

* Procesar los tweets y representar cada tweet como un vector (representar un
documento con un embedding vector).
* Utilice hashing y k vecinos más cercanos para encontrar tweets
que son similares a un tweet determinado.



###  Obteniendo los embeddings de los documentos

#### Modelo de Bag-of-words (BOW) 

Los documentos de texto son secuencias de palabras.
* El orden de las palabras marca la diferencia. Por ejemplo, las frases "La tarta de manzana es mejor que la pizza de pepperoni "y" La pizza de pepperoni es mejor que la tarta de manzana "
tienen significados opuestos debido al orden de las palabras.
* Sin embargo, para algunas aplicaciones, ignorar el orden de las palabras puede permitir
nosotros para formar un modelo eficiente y aún eficaz.
* Este enfoque se denomina modelo de documento de bolsa de palabras.



#### Guardar los vectores de documento en un diccionario



## Buscando los tweets

Ahora tenemos un vector de dimensión (m, d) donde `m` es el número de tweets
(10,000) y `d` es la dimensión de los embeddings (300). 

<a name="3-3"></a>

## Buscando los tweets más similares con LSH

Ahora implementaremos el hashing (LSH) para identificar el tweet más similar.
* En lugar de mirar los 10,000 vectores, puede buscar un subconjunto para encontrar
sus vecinos más cercanos.


<div style="width:image width px; font-size:100%; text-align:center;"><img src='Figures/one.png' alt="alternate text" width="width" height="height" style="width:400px;height:200px;" /> Figure 3 </div>

Puede dividir el espacio vectorial en regiones y buscar dentro de una región los vecinos más cercanos de un vector dado.

<div style="width:image width px; font-size:100%; text-align:center;"><img src='Figures/four.png' alt="alternate text" width="width" height="height" style="width:400px;height:200px;" /> Figure 4 </div>

#### Escogiendo los número de planos

* Cada plano divide el espacio en $ 2 $ partes.
* Entonces $ n $ aviones dividen el espacio en $ 2 ^ {n} $ cubos de hash.
* Queremos organizar 10,000 vectores de documentos en depósitos para que cada depósito tenga vectores de $ ~ 16 $.
* Para eso necesitamos $ \frac {10000} {16} = 625 $ cubos.
* Estamos interesados en $ n $, número de aviones, por lo que $ 2 ^ {n} = 625 $. Ahora, podemos calcular $ n = \log_{2} 625 = 9.29 \approx 10 $.


## Obteniendo el número para el vector hash

Para cada vector, necesitamos obtener un número único asociado a ese vector para poder asignarlo a un "bucket de hash".

### Hyperlanes in vector spaces
* En un espacio vectorial dimensional de $ 3 $, el hiperplano es un plano regular. En el espacio vectorial dimensional de $ 2 $, el hiperplano es una línea.
* Generalmente, el hiperplano es un subespacio que tiene una dimensión $ 1 $ menor que la del espacio vectorial original.
* Un hiperplano se define de forma única por su vector normal.
* El vector normal $ n $ del plano $ \ pi $ es el vector al que todos los vectores en el plano $ \ pi $ son ortogonales (perpendiculares en el caso dimensional de $ 3 $).

### Usando Hiperplanos para cortar el espacio vectorial

Podemos usar un hiperplano para dividir el espacio vectorial en $ 2 $ partes.
* Todos los vectores cuyo producto escalar con el vector normal de un plano es positivo están en un lado del plano.
* Todos los vectores cuyo producto escalar con el vector normal del plano es negativo están en el otro lado del plano.

### Encodeando los buckets hash
* Para un vector, podemos tomar su producto escalar con todos los planos, luego codificar esta información para asignar el vector a un solo cubo hash.
* Cuando el vector apunta al lado opuesto del hiperplano de lo normal, codifíquelo con 0.
* De lo contrario, si el vector está en el mismo lado que el vector normal, codifíquelo por 1.
* Si calcula el producto escalar con cada plano en el mismo orden para cada vector, ha codificado el ID de hash único de cada vector como un número binario, como [0, 1, 1, ... 0].

### Implementando los hash buckets

Inicializamos los hash de la tabla. Es una lista de matrices `N_UNIVERSES`, cada una describe su propia tabla hash. Cada matriz tiene filas `N_DIMS` y columnas` N_PLANES`. Cada columna de esa matriz es un vector normal dimensional `N_DIMS` para cada uno de los hiperplanos` N_PLANES` que se utilizan para crear cubos de la tabla hash particular.

* Primero multiplica tu vector `v`, con un plano correspondiente. Esto le dará un vector de dimensión $ (1, \text{N_planes}) $.
* Luego, convertirá todos los elementos de ese vector en 0 o 1.
* Creas un vector hash haciendo lo siguiente: si el elemento es negativo, se convierte en un 0; de lo contrario, lo cambias a un 1.
* Luego calcula el número único para el vector iterando sobre `N_PLANES`
* Luego multiplica $ 2^i $ por el bit correspondiente (0 o 1).
* Luego almacenará esa suma en la variable `hash_value`.

Cree un hash para el vector en la siguiente función.
Utilice esta fórmula:

$$ hash = \sum_{i=0}^{N-1} \left( 2^{i} \times h_{i} \right) $$

#### Crea los conjuntos de planos
* Cree múltiples (25) conjuntos de planos (los planos que dividen la región).
* Puede pensar en estos como 25 formas distintas de dividir el espacio vectorial con un conjunto diferente de planos.
* Cada elemento de esta lista contiene una matriz con 300 filas (la palabra vector tiene 300 dimensiones) y 10 columnas (hay 10 planos en cada "universo").



## Creando la Tabla Hash

Dado que ya tenemos una representación para cada vector (o tweet), ahora crearemos una tabla hash. Necesitamos una tabla hash, de modo que, dado un hash_id, pueda buscar rápidamente los vectores correspondientes. Esto le permite reducir su búsqueda por una cantidad de tiempo significativa.

<div style="width:image width px; font-size:100%; text-align:center;"><img src='Figures/table.png' alt="alternate text" width="width" height="height" style="width:500px;height:200px;" />  </div>



### Creando todas las tablas hash 



### Approximate K-NN


Implementar aproximadamente K vecinos más cercanos utilizando hash sensible a la localidad,
para buscar documentos que sean similares a un documento dado en el
índice `doc_id`.

##### entradas
* `doc_id` es el índice de la lista de documentos` all_tweets`.
* `v` es el vector de documento para el tweet en` all_tweets` en el índice `doc_id`.
* `planes_l` es la lista de planos (la variable global creada anteriormente).
* `k` es el número de vecinos más cercanos a buscar.
* `num_universes_to_use`: para ahorrar tiempo, podemos usar menos que el total
número de universos disponibles. De forma predeterminada, está configurado en `N_UNIVERSES`,
que es $ 25 $ para esta asignación.

La función `approx_knn` encuentra un subconjunto de vectores candidatos que
están en el mismo "depósito de hash" que el vector de entrada 'v'. Entonces se realiza
los k vecinos más cercanos habituales buscan en este subconjunto (en lugar de buscar
a través de los 10,000 tweets).